In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [104]:
import torch
import numpy as np
import dgl
import pickle

from src.model import get_model, fix_c_gl
from src.dataset import EarlyFixingGraphDataset
from src.net import InstanceGCN

In [19]:
model = get_model(['JUB-58'])

g = dgl.bipartite_from_scipy(model.getA().T, 'var', 'v2c', 'con', 'A')

g.reverse()

Set parameter IntFeasTol to value 1e-09


Graph(num_nodes={'var': 130, 'con': 134},
      num_edges={('con', 'v2c', 'var'): 659},
      metagraph=[('con', 'var', 'v2c')])

In [92]:
A = model.getA()

eta_vars = np.array([v.varname.startswith('eta_n') for v in model.getVars()])

z_cons = A[:,eta_vars].T
x_cons = A[:,~eta_vars].T

g = dgl.heterograph({
    ('z', 'z2c', 'c'): z_cons.nonzero(),
    ('x', 'x2c', 'c'): x_cons.nonzero(),
})
efeats = {
    'z2c': z_cons.data,
    'x2c': x_cons.data,
}
nfeats = {
    'z': np.array(model.getAttr('Obj'))[eta_vars],
    'x': np.array(model.getAttr('Obj'))[~eta_vars],
    'c': model.getAttr('rhs'),
}

In [102]:
with open('../ef_objs.pkl', 'rb') as f:
    ef_objs = pickle.load(f)

ds = EarlyFixingGraphDataset(ef_objs)

(g, nfeats, eweights), y = ds[42]

y

Set parameter IntFeasTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Set parameter IntFeasTol to value 1e-09


-1

In [112]:
gnn = InstanceGCN(2)

nfeats = {k: torch.Tensor(v).double() for k, v in nfeats.items()}

gnn(g, nfeats, eweights)

tensor([[-0.0119]], dtype=torch.float64, grad_fn=<DivBackward0>)

In [82]:
x_cons.data

array([ 1.0000000e+00, -5.0000000e+01, -5.0000000e+01, -5.0000000e+01,
       -5.0000000e+01, -5.0000000e+01, -5.0000000e+01, -5.0000000e+01,
       -5.0000000e+01, -5.0000000e+01, -5.0000000e+01, -6.1110000e+01,
       -6.1110000e+01, -6.1110000e+01, -6.1110000e+01, -6.1110000e+01,
       -6.1110000e+01, -6.1110000e+01, -6.1110000e+01, -6.1110000e+01,
       -6.1110000e+01, -7.2220000e+01, -7.2220000e+01, -7.2220000e+01,
       -7.2220000e+01, -7.2220000e+01, -7.2220000e+01, -7.2220000e+01,
       -7.2220000e+01, -7.2220000e+01, -7.2220000e+01, -8.3330000e+01,
       -8.3330000e+01, -8.3330000e+01, -8.3330000e+01, -8.3330000e+01,
       -8.3330000e+01, -8.3330000e+01, -8.3330000e+01, -8.3330000e+01,
       -8.3330000e+01, -9.4440000e+01, -9.4440000e+01, -9.4440000e+01,
       -9.4440000e+01, -9.4440000e+01, -9.4440000e+01, -9.4440000e+01,
       -9.4440000e+01, -9.4440000e+01, -9.4440000e+01, -1.0556000e+02,
       -1.0556000e+02, -1.0556000e+02, -1.0556000e+02, -1.0556000e+02,
      

In [79]:
z_cons.shape

(20, 134)

In [77]:
x_cons.data.shape

(619,)

In [75]:
rows, cols = x_cons.nonzero()

i = 7
x_cons[rows[i],cols[i]]

-50.0